<a href="https://colab.research.google.com/github/vachkim/PROJECT/blob/master/%EC%9C%A0%ED%86%B5%ED%8C%80/Pro_%EB%B8%94%EB%A1%9C%EA%B7%B8_%ED%82%A4%EC%9B%8C%EB%93%9C_%EB%B9%88%EB%8F%84%EC%88%98_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 코드 설명
- 유명 블로그에서 특정 키워드로 검색했을 때, 몇 건의 결과물이 나오는지 빈도수를 검색하는 함수입니다
- 기간을 설정하는 함수와 빈도수를 검색하는 함수는 별도로 구분했습니다

### 1) 필요 라이브러리 다운

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
import time
from datetime import datetime, timedelta

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import requests
from bs4 import BeautifulSoup as bs

### 2) 크롤링 연습
- 실제 url을 가지고 어떤 항목을 가져와야 빈도수 검색이 가능한지 연습해본 부분입니다
- 안 중요하니 스킵스킵

In [ ]:
url = 'http://search.daum.net/search?w=blog&f=section&SA=daumsec&lpp=10&nil_profile=vsearch&nil_src=blog&q=%ED%8A%B8%EB%9F%AC%ED%94%8C&DA=STC&period=w&sd=20200918153010&ed=20200925153010'

In [ ]:
page = requests.get(url)
soup = bs(page.text,'html.parser')
all = soup.find('span',{'class':'txt_info'})
how_many=all.text

In [ ]:
url = 'http://search.daum.net/search?nil_suggest=btn&w=blog&lpp=10&DA=STC&q={}&sd={}000000&ed={}235959&period=u'

In [ ]:
URL = url.format('트러플',20200918,20200925)
URL

In [ ]:
page = requests.get(URL)
soup = bs(page.text,'html.parser')
all = soup.find('span',{'class':'txt_info'})
how_many=all.text

In [ ]:
how_many

In [ ]:
blank = how_many.split('/')[1].replace(" ","")
maybe = blank.replace("약","")
number = maybe.replace("건","")
nocom = number.replace(",","")
num = int(nocom)
num

### 3) 함수화 진행
- x에 검색 키워드, y에 날짜를 지정하면 해당 날짜에 검색 키워드가 등장한 포스팅의 빈도수를 검색해줍니다
- 우선 빈도수 검색함수는 하루단위로 작동하게 해놨으나 필요하시면 make_end부분의 timedelta(days=1)을 한 달씩 늘도록 수정하면 됩니다
- 다음과 티스토리는 url에서 키워드나 날짜를 입력하는 방식이 동일하기에 코드가 거의 같습니다
- 네이버는 입력받는 형식이 달라서 코드가 조금 다르나 작동하는 원리는 같습니다

In [39]:
#티스토리에서 빈도 검색하는 함수
def get_tistory_frequency(x,y):
    url = 'http://search.daum.net/search?w=blog&f=section&SA=tistory&lpp=10&nil_profile=vsearch&nil_src=blog&q={}&DA=STC&period=u&sd={}000000&ed={}235959'
    make_start = pd.to_datetime(str(y), format='%Y%m%d')
    make_end = make_start +timedelta(days=1)
    z = make_end.strftime("%Y%m%d")
    
    URL = url.format(x,y,z)
    page = requests.get(URL)
    time.sleep(1)
    
    soup = bs(page.text,'html.parser')
    time.sleep(1)
    
    whole = soup.find('div',{'class':'sub_expander'})
    find_number = whole.find('span',{'class':'txt_info'})
    if find_number is None:
        return(0)  
    else:
        found = find_number.text
        split = found.split('/')[1]
        maybe = split.replace("약","")
        number = maybe.replace("건","")
        nocom = number.replace(",","")
        num = int(nocom)
        return(num)

In [102]:
#다음블로그에서 빈도 검색하는 함수
def get_daum_frequency(x,y):
    url = 'http://search.daum.net/search?nil_suggest=btn&w=blog&lpp=10&DA=STC&q={}&sd={}000000&ed={}235959&period=u&f=section&SA=daumsec'
    make_start = pd.to_datetime(str(y), format='%Y%m%d')
    make_end = make_start +timedelta(days=1)
    z = make_end.strftime("%Y%m%d")
    
    URL = url.format(x,y,z)
    page = requests.get(URL)
    time.sleep(1)
    print(URL)
    
    soup = bs(page.text,'html.parser')
    time.sleep(1)
    
    whole = soup.find('div',{'class':'sub_expander'})
    find_number = whole.find('span',{'class':'txt_info'})
    if find_number is None:
        return(0)  
    else:
        found = find_number.text
        split = found.split('/')[1]
        maybe = split.replace("약","")
        number = maybe.replace("건","")
        nocom = number.replace(",","")
        num = int(nocom)
        return(num)

In [140]:
#네이버블로그에서 빈도 검색하는 함수
def get_naver_frequency(x,y):
    url = 'https://search.naver.com/search.naver?where=post&query={}&st=sim&sm=tab_opt&date_from={}&date_to={}&date_option=8&srchby=all&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Ar%2Ca%3Aall%2Cp%3Afrom{}to{}'
    make_start = pd.to_datetime(str(y))
    make_end = make_start +timedelta(days=1)
    z = make_end.strftime('%Y%m%d')

    URL = url.format(x,y,z,y,z)
    page = requests.get(URL)
    time.sleep(1)

    soup = bs(page.text,'html.parser')
    content= soup.find('div',{'class':'blog section _blogBase _prs_blg'})
    whole = content.find('div',{'class':'section_head'})
    result = whole.find('span',{'class':'title_num'})

    if result is None:
        return(0)  
    else:
        found = result.text
        split = found.split('/')[1]
        maybe = split.replace("약","")
        number = maybe.replace("건","")
        nocom = number.replace(",","")
        num = int(nocom)
        return(num)

In [139]:
#네이버 되는지 안된는지 만든 테스트 코드_다음처럼 네이버블로그 홈페이지에서 크롤링하려니 API가 필요하대서 버리고...네이버 검색 후 블로그 섹션으로 들어가 검색
def naver_test(x,y):
    url = 'https://search.naver.com/search.naver?where=post&query={}&st=sim&sm=tab_opt&date_from={}&date_to={}&date_option=8&srchby=all&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Ar%2Ca%3Aall%2Cp%3Afrom{}to{}'
    make_start = pd.to_datetime(str(y))
    make_end = make_start +timedelta(days=1)
    z = make_end.strftime('%Y%m%d')

    URL = url.format(x,y,z,y,z)
    page = requests.get(URL)
    time.sleep(1)

    soup = bs(page.text,'html.parser')

    content= soup.find('div',{'class':'blog section _blogBase _prs_blg'})
    whole = content.find('div',{'class':'section_head'})
    result = whole.find('span',{'class':'title_num'})

    if result is None:
        return(0)  
    else:
        found = result.text
        split = found.split('/')[1]
        maybe = split.replace("약","")
        number = maybe.replace("건","")
        nocom = number.replace(",","")
        num = int(nocom)
        return(num)

In [42]:
#YYYYMMDD형식으로 날짜 리스트 만드는 함수 = 다음, 티스토리 검색용
def get_date_in_line(year,month,day,x):
    start_date = date(year,month,day)
    day_list = []
    for how_many in range(1,x+1):
        formattedDate = start_date.strftime("%Y%m%d")
        day_list.append(formattedDate)
        start_date += timedelta(days=1)
    return(day_list)

In [43]:
#추가로 만든 YYYY-MM-DD형식으로 날짜 리스트 만드는 함수 
def get_date_with_split(year,month,day,x):
    start_date = date(year,month,day)
    day_list = []
    for how_many in range(1,x+1):
      formattedDate = start_date.strftime('%Y-%m-%d')
      day_list.append(formattedDate)
      start_date += timedelta(days=1)
    return(day_list)

### 4) 결과값 확인
- start_list를 통해 다음블로그와 티스토리의 빈도수

In [ ]:
start_list = get_date_in_line(2018,7,8,365)
start_list

In [56]:
daum_frequency=[]

for day in start_list:
    start = int(day)
    daum_frequency.append(get_daum_frequency('트러플',start))

In [57]:
tistory_frequency=[]

for day in start_list:
    start = int(day)
    tistory_frequency.append(get_tistory_frequency('트러플',start))

In [141]:
naver_frequency = []

for day in start_list:
  start = int(day)
  naver_frequency.append(get_naver_frequency('트러플',start))

### 5) 데이터프레임 생성 및 저장
- 크롤링한 데이터를 바탕으로 데이터 프레임을 만들고 저장합니다 

In [143]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Mounted at /content/drive


In [145]:
%cd drive/'My Drive'/'아시아경제 유통팀'/'조사자료'

/content/drive/My Drive/아시아경제 유통팀/조사자료


In [146]:
%ls

'오프라인 년도별 비교'/
'전자상거래 이용 현황과 구매 행태 (1).pdf'
'유통산업 현황 2019 및 2020 산업전망 보고서.pdf'
 온라인으로_식품을_구매하는_장소_20200922182431.csv
'2020년 7월 온라인쇼핑 동향.gdoc'
'2020년 7월 온라인쇼핑 동향.pdf'
 2020년만.pdf
 산업통상자원부유통.csv
'오프라인 카테고리 분석.pdf'
 Transaction_value_of_Online_shopping_mall_by_commodity_groups_coverage_for_goods_20200923102902.csv
 Transaction_value_of_Online_shopping_mall_by_commodity_groups_coverage_for_goods_20200923102902.gsheet
 신한카드데이터.xlsx
 국민카드데이터.xlsx
 신용카드증가데이터.xlsx
 트러플네이버리뷰빈도수.xlsx


In [160]:
Blog_Frequency = pd.DataFrame({'Naver':naver_frequency,'Daum':daum_frequency,'Tistory':tistory_frequency},index=start_list)
Blog_Frequency

,Naver,Daum,Tistory
20180708,249,0,3
20180709,276,1,4
20180710,254,1,3
20180711,226,2,2
20180712,210,2,4
...,...,...,...
20190703,501,0,3
20190704,484,0,6
20190705,385,0,6
20190706,336,0,6


In [161]:
from google.colab import files

Blog_Frequency.to_csv('블로그리뷰.csv')
files.download('블로그리뷰.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>